In [30]:
import pandas as pd
import psycopg2
import os
from bs4 import BeautifulSoup 
from IPython.display import display

In [2]:
def conexion():
    baseDatos = os.getenv('baseDatos')
    conn = psycopg2.connect(baseDatos)
    cursor = conn.cursor()
    return conn, cursor

In [8]:
conn, cursor = conexion()
query = '''SELECT cuenta, CONCAT_WS(' ',nombre,p_apellido,s_apellido)
            FROM becas.alumnos
            WHERE apoyo_alimenticio IS TRUE;'''
cursor.execute(query)
datos = cursor.fetchall()
datos_df = pd.DataFrame(datos, columns=['cuenta','nombre'])
conn.close()
cursor.close()
datos_df.head()

In [9]:
import sys
import os
sys.path.append('/Users/miguelhernandez/Library/CloudStorage/OneDrive-EscuelaNacionalColegiodeCienciasyHumanidades,UNAM/2024-1/secretaria_estudiantil/Pruebas alimentacion de pagina/')
from scrapping_mini import Scraper, Alumno

In [18]:
datos_df_prueba = datos_df[:1].copy()
datos_df_prueba
datos_df_prueba['tutor'] = None
datos_df_prueba['correo_tutor'] = None

In [ ]:
from bs4 import BeautifulSoup

# Iniciar sesión en el scraper
scraper = Scraper(usuario="ROSE631227A85", contraseña="844346")
scraper.login()

# Iterar sobre las filas del DataFrame 'datos_df'
for index, alumno in datos_df.iterrows():
    cuenta_alumno = alumno['cuenta']
    
    # Obtener la página del alumno usando la cuenta
    html_alumno = scraper.obtener_pagina_alumno(cuenta_alumno)
    
    # Analizar el HTML con BeautifulSoup
    soup = BeautifulSoup(html_alumno, 'html.parser')
    
    try:
        # Encontrar la fila que contiene 'Tutor:'
        tutor_row = None
        for tr in soup.find_all('tr'):
            th = tr.find('th')
            if th and 'Tutor:' in th.text:
                tutor_row = tr
                break
        
        # Extraer el nombre y correo del tutor
        if tutor_row:
            th_elements = tutor_row.find_all('th')
            if len(th_elements) >= 3:
                tutor_info = th_elements[2]
                # Obtener los contenidos separados del 'th' (maneja automáticamente los <br/>)
                tutor_contents = tutor_info.contents
                # Filtrar los contenidos que sean cadenas de texto
                tutor_texts = [content.strip() for content in tutor_contents if isinstance(content, str)]
                # Asignar nombre y correo
                tutor_name = tutor_texts[0] if len(tutor_texts) > 0 else ''
                tutor_email = tutor_texts[1] if len(tutor_texts) > 1 else ''
            else:
                tutor_name = ''
                tutor_email = ''
        else:
            tutor_name = ''
            tutor_email = ''
    except Exception as e:
        # En caso de error, asignar valores vacíos
        tutor_name = ''
        tutor_email = ''
    
    # Agregar los datos al DataFrame
    datos_df.at[index, 'tutor'] = tutor_name
    datos_df.at[index, 'correo_tutor'] = tutor_email
    
    # Imprimir los resultados
    print(f"Tutor: {tutor_name}")
    print(f"Correo del tutor: {tutor_email}")

# Cerrar el scraper
scraper.cerrar()

In [36]:
import pickle
# Crear un diccionario que contenga los tres DataFrames
dataframes = {
    'con_tutor': con_tutor,
    'sin_tutor': sin_tutor,
    'datos_df': datos_df
}

# Guardar el diccionario en un archivo pickle
with open('dataframes.pkl', 'wb') as f:
    pickle.dump(dataframes, f)

### Los dataframes son con_tutor, sin_tutor, datos_df

In [3]:
import pickle

# Cargar el archivo pickle
with open('dataframes.pkl', 'rb') as f:
    dataframes = pickle.load(f)

In [4]:
con_tutor = dataframes['con_tutor']
sin_tutor = dataframes['sin_tutor']

In [7]:
conteo_alumnos_por_tutor = con_tutor.groupby('correo_tutor').size().reset_index(name='cantidad_alumnos')

In [8]:
print(conteo_alumnos_por_tutor['cantidad_alumnos'].mean())

In [9]:
len(conteo_alumnos_por_tutor)

In [14]:
conn, cursor = conexion()

In [16]:
con_tutor.head(0)

In [22]:
conn, cursor = conexion()
try:
    # Iterar sobre los alumnos para realizar la inserción
    for index, alumno in con_tutor.iterrows():
        cuenta = alumno['cuenta']  # Asegúrate de que este valor sea un entero
        tutor = alumno['tutor']
        correo_tutor = alumno['correo_tutor']
        
        try:
            # Ejecutar la consulta SQL
            cursor.execute(query, (cuenta, tutor, correo_tutor))
        except Exception as e:
            # Si algo falla, hacer rollback
            conn.rollback()
            print(f"Error al insertar en la cuenta {cuenta}: {e}")
        else:
            # Si no hay errores, hacer commit
            conn.commit()
finally:
    # Cerrar cursor y conexión en todos los casos, después del bucle
    cursor.close()
    conn.close()

In [5]:
tutores = con_tutor[['tutor', 'correo_tutor']].drop_duplicates().reset_index(drop=True)

In [8]:
import string 
import random

In [10]:
def generar_identificador_tutor(longitud=15):
    # Definir los caracteres que se utilizarán para generar el identificador
    caracteres = string.ascii_letters + string.digits + "!#$%^&*_+="
    # Generar una cadena aleatoria de la longitud especificada
    identificador = ''.join(random.choice(caracteres) for _ in range(longitud))
    return identificador

In [12]:
tutores['identificador']=None
tutores['id']=None

In [26]:
query = '''
INSERT INTO becas.tutores (nombre, correo, identificador)
VALUES (%s, %s, %s)
RETURNING id
'''

In [27]:
print(query)

In [28]:
conn, cursor = conexion()

try:
    for _, tutor in tutores.iterrows():
        try:
            nombre = tutor['tutor']
            correo_tutor = tutor['correo_tutor']
            identificador = generar_identificador_tutor()

            # Consulta para insertar el tutor en la base de datos
            cursor.execute(query, (nombre, correo_tutor, identificador))

            # Obtener el id del tutor generado
            tutor_id = cursor.fetchone()[0]

            # Actualizar el DataFrame con el id y el identificador
            tutores.at[_, 'id'] = tutor_id
            tutores.at[_, 'identificador'] = identificador

            # Confirmar la transacción
            conn.commit()

        except Exception as e:
            print(f'Ocurrió el siguiente error al procesar el tutor {nombre}: {e}')
            conn.rollback()  # Hacer rollback si ocurre un error en la inserción

finally:
    # Asegurarse de cerrar la conexión y el cursor fuera del ciclo
    cursor.close()
    conn.close()

In [33]:
display(con_tutor.columns)
display(tutores.columns)

In [34]:
base_alumnos = pd.merge(con_tutor, tutores, on=['tutor', 'correo_tutor'], how='left')

In [37]:
base_alumnos.head(1)

In [38]:
query1 = '''UPDATE becas.apoyo_alimenticio
            SET tutor=%s
            WHERE cuenta=%s;'''

conn, cursor = conexion()

for _, alumno in base_alumnos.iterrows():
    cuenta = alumno['cuenta']
    tutor = alumno['id']
    try:
        cursor.execute(query1, (tutor, cuenta))
    except Exception as e:
        print(f'Ocurrió el siguiente error: {e}')
        
# Confirmar los cambios
conn.commit()

# Cerrar cursor y conexión
cursor.close()
conn.close()